In [2]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import numpy as np

In [3]:
model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('ohe_geo.pkl','rb') as file:
    ohe_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [24]:
input_data = {
    'credit_score':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [25]:
data = pd.DataFrame([input_data])
data

,credit_score,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [26]:
geo = ohe_geo.transform(data[['Geography']])

In [27]:
geo.toarray()

array([[1., 0., 0.]])

In [28]:
data['Gender'] = label_encoder_gender.transform(data['Gender'])

In [29]:
ohe_geo.get_feature_names_out

<bound method OneHotEncoder.get_feature_names_out of OneHotEncoder()>

In [30]:
geography = pd.DataFrame(geo.toarray(),columns=ohe_geo.get_feature_names_out(['Geography']))

In [31]:
data = pd.concat([data.drop('Geography',axis=1),geography],axis=1)

In [32]:
data

,credit_score,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [33]:
data.rename(columns={'credit_score': 'CreditScore'}, inplace=True)

In [35]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
data_scaled = scaler.transform(data)

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [38]:
prediction = model.predict(data_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


array([[0.02976235]], dtype=float32)

In [40]:
prediction_probab = prediction[0][0]

In [42]:
if prediction_probab > 0.5:
    print(1)
else:
    print(0)

0
